In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ATNLP 
%pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ATNLP


'/content/drive/MyDrive/ATNLP'

In [ ]:
!pip install wandb -Uq

     |████████████████████████████████| 1.9 MB 8.0 MB/s 
     |████████████████████████████████| 182 kB 54.5 MB/s 
     |████████████████████████████████| 174 kB 42.4 MB/s 
     |████████████████████████████████| 62 kB 930 kB/s 
     |████████████████████████████████| 173 kB 45.8 MB/s 
     |████████████████████████████████| 168 kB 53.2 MB/s 
     |████████████████████████████████| 168 kB 10.4 MB/s 
     |████████████████████████████████| 166 kB 38.6 MB/s 
     |████████████████████████████████| 166 kB 38.9 MB/s 
     |████████████████████████████████| 162 kB 34.4 MB/s 
     |████████████████████████████████| 162 kB 48.2 MB/s 
     |████████████████████████████████| 158 kB 39.3 MB/s 
     |████████████████████████████████| 157 kB 47.3 MB/s 
     |████████████████████████████████| 157 kB 46.3 MB/s 
     |████████████████████████████████| 157 kB 48.8 MB/s 
     |████████████████████████████████| 157 kB 45.2 MB/s 
     |████████████████████████████████| 157 kB 48.9 MB/s 
     |███████████

In [ ]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import spacy
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.nn.utils import clip_grad_value_
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


### Loading data files

SCAN


In [ ]:
SOS_token = 0
EOS_token = 1

class Format:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {SOS_token: "SOS", EOS_token: "EOS"}
        # self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.index2word[self.n_words] = word
            self.n_words += 1
            self.word2count[word] = 1
        else:
            self.word2count[word] += 1

In [ ]:
def readFile(filename):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('SCAN/%s.txt' % filename, encoding='utf-8').read().strip().split('\n')

    # exclude "IN: "
    pairs = [s[4:].split(' OUT: ') for s in lines]

    input_lang = Format("input")
    output_lang = Format("output")

    return input_lang, output_lang, pairs

In [ ]:
def prepareData(filename):
    input_lang, output_lang, pairs = readFile(filename)
    print("Read %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [ ]:
input_lang, output_lang, pairs = prepareData("simple_split/tasks_train_simple")
input_lang_test, output_lang_test, pairs_test = prepareData("simple_split/tasks_test_simple")
print(random.choice(pairs))
print(len(pairs), type(pairs))

Reading lines...


FileNotFoundError: ignored

In [ ]:
input_lang

### Models （re-implementations)

##### Overall-best

2-layer LSTM with 200 hidden units per layer, no attention, and dropout applied at the 0.5 level

In [ ]:
class Encoder(nn.Module):
    def __init__(self, 
                 input_dim, 
                 emb_dim, 
                 hid_dim, 
                 n_layers, 
                 dropout):
      
        super().__init__()
        self.hid_dim = hid_dim
        self.n_layers = n_layers   
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the  tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

##### Experiment 1

LSTM with no attention, 2 layers of 200 hidden units, and no dropout.

In [ ]:
class EncoderLSTM(nn.Module):
    def __init__(self, 
                 input_size, 
                 hidden_size, 
                 dropout_prob=0.0):
      
        super(EncoderLSTM, self).__init__()

        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=2, dropout=dropout_prob)
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, input, hidden):

        # embedded = [1, 1, emb dim]
        embedded = (self.dropout(self.embedding(input))).view(1, 1, -1)
        output = embedded
        
        ## hidden is a tuple (hidden, cell)
        output, hidden = self.lstm(output, hidden)
        return output, hidden

    def initHidden(self):
        return (torch.zeros(2, 1, self.hidden_size, device=device), 
                torch.zeros(2, 1, self.hidden_size, device=device))

In [ ]:
class DecoderLSTM(nn.Module):

    # add dropout
    def __init__(self, hidden_size, output_size, dropout_prob=0.0):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=2, dropout=dropout_prob)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_prob)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = self.dropout(output)

        output = F.relu(output)
        output, hidden = self.lstm(output, hidden)

        # ? paramter sharing 
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return (torch.zeros(2, 1, self.hidden_size, device=device), torch.zeros(2, 1, self.hidden_size, device=device))


### Training and Evaluation of the Models

#### Training

In [ ]:
# obtain word indices in a sentence
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


# transform to tensor format and add a special token
def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

# for input and target (one-hot vectors)
def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [ ]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    loss = 0

    # put each word into encoder at time t intead of a sentence togther
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    clip_grad_value_(encoder.parameters(), 5)
    clip_grad_value_(decoder.parameters(), 5)

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.001):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)


    # Note that, in all experiments, the number of distinct training commands is well below 100k: we randomly sampled them with replacement to reach the target size (100,000)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]

    # loss function
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        
        ### Print the loss and Time
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))
            
        if iter % plot_every == 0:
            wandb.log({"loss": plot_loss_total / plot_every})
            # plot_loss_avg = plot_loss_total / plot_every
            # plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
    # showPlot(plot_losses)

#### Evaluation

In [ ]:
def evaluate(encoder, decoder, sentence):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        while(True):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [ ]:
def accuracy(encoder, decoder, pairs):
    sum = 0
    i = 0
    for pair in pairs:
        i+=1
        # print("%.2f" % (i/len(pairs))) 
        output_words = evaluate(encoder, decoder, pair[0])[0:-1]
        output_sentence = ' '.join(output_words)
        if output_sentence == pair[1]:
          sum +=1
    wandb.log({"accuracy": sum/len(pairs)})
    return sum/len(pairs)

### Experiments

Fixed length


In [ ]:
for run in range(2):
    wandb.init(
        project="ATNLP",
        name = f"Exp1_simple_Exp1_{run+3}",
        config={
            "hidden_size": 200,
            "dropout_prob": 0,
            })
    
    # Copy your config 
    config = wandb.config

    print("Training:")
    exp1_encoder = EncoderLSTM(input_lang.n_words, config.hidden_size, config.dropout_prob).to(device)
    exp1_decoder = DecoderLSTM(config.hidden_size, output_lang.n_words).to(device)
    trainIters(exp1_encoder, exp1_decoder, 100000, print_every= 2000, plot_every= 2000)

    print("Testing:") 
    accuracy(exp1_encoder, exp1_decoder, pairs_test)

wandb: Currently logged in as: yuqinzhou. Use `wandb login --relogin` to force relogin


Training:
0m 46s (- 37m 41s) (2000 2%) 1.0767
1m 25s (- 34m 2s) (4000 4%) 0.4964
2m 3s (- 32m 16s) (6000 6%) 0.3091
2m 44s (- 31m 35s) (8000 8%) 0.2175
3m 23s (- 30m 29s) (10000 10%) 0.1518
4m 2s (- 29m 38s) (12000 12%) 0.1254
4m 40s (- 28m 44s) (14000 14%) 0.0875
5m 20s (- 28m 0s) (16000 16%) 0.0775
5m 59s (- 27m 17s) (18000 18%) 0.0677
6m 38s (- 26m 33s) (20000 20%) 0.0456
7m 17s (- 25m 52s) (22000 22%) 0.0339
7m 57s (- 25m 12s) (24000 24%) 0.0290
8m 38s (- 24m 36s) (26000 26%) 0.0295
9m 17s (- 23m 54s) (28000 28%) 0.0308
9m 57s (- 23m 13s) (30000 30%) 0.0199
10m 36s (- 22m 31s) (32000 32%) 0.0239
11m 15s (- 21m 52s) (34000 34%) 0.0148
11m 54s (- 21m 10s) (36000 36%) 0.0110
12m 34s (- 20m 30s) (38000 38%) 0.0111
13m 13s (- 19m 50s) (40000 40%) 0.0123
13m 54s (- 19m 12s) (42000 42%) 0.0163
14m 33s (- 18m 32s) (44000 44%) 0.0121
15m 12s (- 17m 51s) (46000 46%) 0.0142
15m 52s (- 17m 11s) (48000 48%) 0.0050
16m 31s (- 16m 31s) (50000 50%) 0.0130
17m 10s (- 15m 50s) (52000 52%) 0.0080
17m

accuracy,▁
loss,█▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.99904
loss,0.01095


Training:
0m 45s (- 36m 55s) (2000 2%) 1.0686
1m 23s (- 33m 12s) (4000 4%) 0.5013
2m 1s (- 31m 48s) (6000 6%) 0.3199
2m 40s (- 30m 50s) (8000 8%) 0.2065
3m 22s (- 30m 22s) (10000 10%) 0.1571
4m 2s (- 29m 36s) (12000 12%) 0.1127
4m 41s (- 28m 47s) (14000 14%) 0.0732
5m 20s (- 28m 1s) (16000 16%) 0.0732
5m 59s (- 27m 15s) (18000 18%) 0.0463
6m 37s (- 26m 31s) (20000 20%) 0.0409
7m 17s (- 25m 50s) (22000 22%) 0.0315
7m 56s (- 25m 7s) (24000 24%) 0.0243
8m 37s (- 24m 33s) (26000 26%) 0.0201
9m 16s (- 23m 52s) (28000 28%) 0.0226
9m 55s (- 23m 9s) (30000 30%) 0.0143
10m 34s (- 22m 28s) (32000 32%) 0.0109
11m 13s (- 21m 47s) (34000 34%) 0.0103
11m 52s (- 21m 7s) (36000 36%) 0.0134
12m 31s (- 20m 25s) (38000 38%) 0.0118
13m 10s (- 19m 45s) (40000 40%) 0.0040
13m 50s (- 19m 7s) (42000 42%) 0.0105
14m 30s (- 18m 28s) (44000 44%) 0.0079
15m 10s (- 17m 48s) (46000 46%) 0.0090
15m 49s (- 17m 8s) (48000 48%) 0.0068
16m 28s (- 16m 28s) (50000 50%) 0.0057
17m 7s (- 15m 48s) (52000 52%) 0.0078
17m 46s 